### Shared Word File Process

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "Spanish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 28  # 28  # native word end index

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # always must be False in this part
native_word = True # True for adding native word
etymology_word = True  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.
print(f"{file_ext}")

1


In [4]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)

### Twogram In Threegram

In [5]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988212,karneleme,5
988213,karnaya,5
988214,dörtlümüzün,5
988215,karnavalınız,5


In [6]:
df_shared_file = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency{file_ext}.xlsx")
#df_shared_file = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx")
df_shared_file

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,bir,18835735.0,bir şey,859944,bir şey yok,113165.0,bir şey var mı,41773.0,komik bir şey mi var,901.0,ne var,62532.0
1,bu,11062659.0,değil mi,585879,bir şey var,110455.0,bir şey daha var,12185.0,bir şey yok değil mi,893.0,ben de,59972.0
2,ne,8025880.0,ben de,377765,bu da ne,89463.0,bir şey mi var,11834.0,o kadar çok şey var,790.0,değil mi,58386.0
3,ve,7766036.0,var mı,281902,bir şey değil,68498.0,ne var ne yok,9276.0,bir şey yok bir şey,490.0,bu da ne,50098.0
4,için,5484109.0,ne kadar,237876,bir şey mi,64933.0,bir şey değil mi,5080.0,ama bir şey daha var,393.0,ben mi,33652.0
...,...,...,...,...,...,...,...,...,...,...,...,...
14592,NaN,NaN,kadar favori,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14593,NaN,NaN,bir otobiyografi,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14594,NaN,NaN,şey oksijen,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14595,NaN,NaN,bir otomotiv,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

In [8]:
def word_in_wordgroup(df, list_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, list_column and target_column are 
       dataframe column string name. list_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{list_column}"].dropna():
        try:
            #word_in_twogram = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(10)  # Option
            word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(100) 
        except:
            pass        
        word_in_word_cluster.insert(0,f"{list_column}",i)
        df_word_result = pd.concat([df_word_result,word_in_word_cluster], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [9]:
#df_shared_count = word_count_result(df_shared_file,["threegram"])
#df_shared_count

In [10]:
df_shared_file["twogram"].nunique()

14597

In [11]:
df_two_in_three = word_in_wordgroup(df_shared_file, "twogram", "threegram")
df_two_in_three

,twogram,threegram
0,bir şey,bir şey yok
1,bir şey,bir şey var
2,bir şey,bir şey değil
3,bir şey,bir şey mi
4,bir şey,bir şey daha
...,...,...
21981,bunu romantik,bunu romantik bir
21982,bir nörolog,bir nörolog var
21983,kadar fantastik,ne kadar fantastik
21984,kadar fantastik,bu kadar fantastik


In [12]:
df_two_in_three["twogram"].nunique()

5493

In [13]:
df_shared_select_twogram = df_shared_file.loc[:,["twogram","freq_twogram"]]
df_shared_select_twogram

,twogram,freq_twogram
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,var mı,281902
4,ne kadar,237876
...,...,...
14592,kadar favori,6
14593,bir otobiyografi,6
14594,şey oksijen,6
14595,bir otomotiv,6


In [14]:
set_shared_twogram = set(df_shared_select_twogram["twogram"])
set_two_three = set(df_two_in_three["twogram"])

In [15]:
df_twogram_in_threegram = pd.DataFrame(set_two_three, columns=["twogram"])  # columns=["twogram_in_threegram"]
df_twogram_in_threegram

,twogram
0,ve banka
1,daha ama
2,bu operasyon
3,enteresan değil
4,milyon ton
...,...
5488,antika bir
5489,daha popülist
5490,için röportaj
5491,kamp çok


In [16]:
df_twogram_in_threegram_freq = pd.merge(df_twogram_in_threegram, df_shared_select_twogram, how="left", on="twogram")
df_twogram_in_threegram_freq.sort_values(by="freq_twogram", ascending=False, inplace=True)
df_twogram_in_threegram_freq.rename(columns={"twogram":"twogram_in_threegram","freq_twogram":"freq_two_in_three"}, inplace=True)
df_twogram_in_threegram_freq.reset_index(drop=True, inplace=True)
df_twogram_in_threegram_freq

,twogram_in_threegram,freq_two_in_three
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,var mı,281902
4,ne kadar,237876
...,...,...
5488,çok puan,6
5489,dijital fotoğraf,6
5490,kadar fantastik,6
5491,de hamburger,6


In [17]:
df_twogram_diff = pd.DataFrame(set_shared_twogram.difference(set_two_three), columns=["twogram"])
df_twogram_diff

,twogram
0,spor çok
1,beni kale
2,amerikan pilot
3,barones bir
4,şey bisiklet
...,...
9099,ve puding
9100,polis barikat
9101,poker gibi
9102,seni deşifre


In [18]:
df_twogram_diff_freq = pd.merge(df_twogram_diff, df_shared_select_twogram, how="left", on="twogram")
df_twogram_diff_freq.sort_values(by="freq_twogram", ascending=False, inplace=True)
df_twogram_diff_freq.reset_index(drop=True, inplace=True)
df_twogram_diff_freq

,twogram,freq_twogram
0,evet kaptan,1357
1,evet madam,1129
2,ve motor,871
3,seni manyak,643
4,şifre ne,543
...,...,...
9099,şövalye o,6
9100,kabine var,6
9101,ben barbekü,6
9102,gibi karantina,6


In [19]:
df_shared_file["twogram"] = df_twogram_diff_freq["twogram"]
df_shared_file["freq_twogram"] = df_twogram_diff_freq["freq_twogram"]

In [20]:
df_shared_twogram_process = pd.concat([df_shared_file,df_twogram_in_threegram_freq], axis=1)
df_shared_twogram_process

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence,twogram_in_threegram,freq_two_in_three
0,bir,18835735.0,evet kaptan,1357.0,bir şey yok,113165.0,bir şey var mı,41773.0,komik bir şey mi var,901.0,ne var,62532.0,bir şey,859944.0
1,bu,11062659.0,evet madam,1129.0,bir şey var,110455.0,bir şey daha var,12185.0,bir şey yok değil mi,893.0,ben de,59972.0,değil mi,585879.0
2,ne,8025880.0,ve motor,871.0,bu da ne,89463.0,bir şey mi var,11834.0,o kadar çok şey var,790.0,değil mi,58386.0,ben de,377765.0
3,ve,7766036.0,seni manyak,643.0,bir şey değil,68498.0,ne var ne yok,9276.0,bir şey yok bir şey,490.0,bu da ne,50098.0,var mı,281902.0
4,için,5484109.0,şifre ne,543.0,bir şey mi,64933.0,bir şey değil mi,5080.0,ama bir şey daha var,393.0,ben mi,33652.0,ne kadar,237876.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14592,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14593,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14594,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14595,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
df_shared_twogram_process.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency{file_ext}2.xlsx", index=False)

##### Concat Result With Comma

In [22]:
df_word_order_twogram = word_in_wordgroup(df_shared_twogram_process, "word", "twogram")
df_word_order_threegram = word_in_wordgroup(df_shared_twogram_process, "word", "threegram") 
df_word_order_fourgram = word_in_wordgroup(df_shared_twogram_process, "word", "fourgram") 
df_word_order_fivegram = word_in_wordgroup(df_shared_twogram_process, "word", "fivegram")
df_word_order_sentence = word_in_wordgroup(df_shared_twogram_process, "word", "sentence")
df_word_order_twogram_threegram = word_in_wordgroup(df_shared_twogram_process, "word", "twogram_in_threegram") 

In [23]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["word"])["sentence"].apply(", ".join).reset_index()
df_word_order_join_twogram_threegram = df_word_order_twogram_threegram.groupby(["word"])["twogram_in_threegram"].apply(", ".join).reset_index()

In [24]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence,df_word_order_join_twogram_threegram]

In [25]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all  

,word,twogram,threegram,fourgram,fivegram,sentence,twogram_in_threegram
0,abaküs,bir abaküs,NaN,NaN,NaN,bu bir abaküs,NaN
1,abluka,bir abluka,NaN,NaN,NaN,"bu bir abluka, abluka mı",NaN
2,absorbe,onu absorbe,NaN,NaN,NaN,NaN,NaN
3,acente,"bu acente, acente beni, bir acente",NaN,NaN,NaN,NaN,NaN
4,adaptasyon,"var adaptasyon, adaptasyon için, ve adaptasyon...",bir adaptasyon değil,"bir adaptasyon değil mi, mutasyon ve adaptasyo...",NaN,NaN,"adaptasyon değil, bir adaptasyon"
...,...,...,...,...,...,...,...
1317,metabolizma,NaN,NaN,NaN,NaN,metabolizma ne,NaN
1318,nanoteknoloji,NaN,NaN,NaN,NaN,nanoteknoloji mi,NaN
1319,rulet,NaN,NaN,NaN,NaN,"rulet mi, rulet gibi",NaN
1320,süpersonik,NaN,NaN,NaN,NaN,bu süpersonik,NaN


In [26]:
# Option For Word Frequency
if shared_word_frequency:
    df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="inner", on="word")
    df_word_order_join_all.drop_duplicates(inplace=True)
    df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram","fourgram","fivegram","sentence","twogram_in_threegram"]]
    df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
    df_word_order_join_all.reset_index(drop=True, inplace=True)
else:
    pass

df_word_order_join_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence,twogram_in_threegram
0,bir,18835735,"bir petrol, bir sperm, bir stres, müslüman bir...","bir şey yok, bir şey var, bir şey değil, bir ş...","bir şey var mı, bir şey daha var, bir şey mi v...","komik bir şey mi var, bir şey yok değil mi, bi...","bir şey değil, yok bir şey, bir şey yok, bir ş...","bir şey, bu bir, bir daha, ben bir, bana bir, ..."
1,bu,11062659,"bu ajan, bu kaptan, bu park, bu profesör, moda...","bu da ne, bu kadar mı, ve bu da, bu kadar çok,...","bu şey de ne, bu o kadar da, bir şey değil bu,...","bu da bir şey mi, bu daha bir şey değil, bu no...","bu da ne, bu ne, bu o, bu kadar, bu kadar mı, ...","bu kadar, bu da, bu çok, bu bir, bu ne, ama bu..."
2,ne,8025880,"şifre ne, ne takım, ne bebek, ne trajedi, ne ş...","bu da ne, ne kadar da, o da ne, daha ne kadar,...","ne var ne yok, bu şey de ne, daha ne kadar var...","ne kadar çok o kadar, ne sen ne de ben, ne kad...","ne var, bu da ne, ne için, bu ne, ne kadar, ne...","ne kadar, ne var, da ne, bu ne, sen ne, ne içi..."
3,ve,7766036,"ve motor, argo ve, ve disiplin, ve park, mutan...","sen ve ben, ve ben de, ve sen de, ve bu da, ve...","ve bir şey daha, ve sen de bunu, ve ben de onu...","ve bir şey daha var, ve bir o kadar da, ve bir...","sen ve ben, ve sen, ve ben, ve bu, ve ne, ve b...","ve ben, ve bu, ve bir, sen ve, ve sen, ve o, v..."
4,için,5484109,"için kontrol, için polis, için onur, bomba içi...","için bir şey, için ne kadar, bir şey için, içi...","için bir şey var, için çok ama çok, için daha ...","için bir şey var mı, için bir şey mi var, için...","ne için, bu ne için, o ne için, şey için, ne i...","için bir, için çok, ne için, için mi, için ne,..."
...,...,...,...,...,...,...,...,...
1317,inovasyon,18,ve inovasyon,NaN,NaN,NaN,NaN,NaN
1318,varyant,18,"yok varyant, varyant yok",NaN,NaN,NaN,polis yok varyant yok,NaN
1319,nosyon,17,değil nosyon,NaN,NaN,NaN,NaN,NaN
1320,refüze,13,seni refüze,NaN,NaN,NaN,NaN,NaN


In [27]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency{file_ext}2.xlsx", index=False)

#### Copy Move And Delete

In [28]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*{file_ext}2.xlsx")
output_file

['Turkish_Spanish_Shared_Result_With_Frequency12.xlsx',
 'Turkish_Spanish_Shared_Join_Result_Without_Frequency12.xlsx']

In [29]:
for k in output_file:
    source = k # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [30]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass

### Prefix Suffix Result Analysis

In [69]:
import os
import multiprocessing
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil

In [70]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "Spanish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# word sample
word_sample = True  # True, False
word_sample_num = 20

# shared word frequency
shared_word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = True  # True, False  # always must be True in this part
native_word = True # True for adding native word
etymology_word = False  # True for adding etymology word

# adding output file extention
if (not prefix_suffix) & etymology_word & native_word:
    file_ext = "1"
elif (not prefix_suffix) & etymology_word & (not native_word):
    file_ext = "2"
elif prefix_suffix & etymology_word & native_word:
    file_ext = "3"
elif prefix_suffix & etymology_word & (not native_word):
    file_ext = "4"
elif prefix_suffix & (not etymology_word) & native_word:
    file_ext = "5"
elif (not prefix_suffix) & (not etymology_word) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              
# 1 => for native word and etymology word without prefix suffix. 
# 2 => for only etymology word without prefix suffix. 
# 3 => for native word and etymology word with prefix suffix. prefix_suffix, native_word and etymology_word must be True. 
# 4 => for only etymology word with prefix suffix.
# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.
print(f"{file_ext}")

5


In [71]:
def word_in_wordgroup(df, source_column, target_column):

    '''word_in_wordgroup(df, "word", "twogram"):
       df is dataframe, source_column and target_column are 
       dataframe column string name. source_column convert list
       values that are in target column.
    '''
    
    df_select = df[[f"{target_column}"]].dropna()
    df_word_result = pd.DataFrame()
    for i in df[f"{source_column}"].dropna():
        try:
            if word_sample:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)].head(word_sample_num)  # Option
            else:
                word_in_word_cluster = df_select[df_select[f"{target_column}"].str.contains(fr"(?:\s|^){i}(?:\s|$)", na=True)] 
        except:
            pass        
        word_in_word_cluster.insert(0,f"{source_column}",i)
        df_word_result = pd.concat([df_word_result,word_in_word_cluster], axis=0)
    df_word_result.reset_index(drop=True, inplace=True)

    return df_word_result

In [72]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988212,karneleme,5
988213,karnaya,5
988214,dörtlümüzün,5
988215,karnavalınız,5


In [73]:
df_word_prefix_suffix = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
df_word_prefix_suffix = df_word_prefix_suffix[["search_word","word"]]
df_word_prefix_suffix

,search_word,word
0,ama,ama
1,bana,bana
2,ben,ben
3,ben,bence
4,ben,bende
...,...,...
162,şey,şeyler
163,şey,şeylerden
164,şey,şeylere
165,şey,şeyleri


In [74]:
df_ety_prefix_suffix = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Result/{lang_folder.capitalize()}/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Word_Prefix_Suffix_Custom_Result.xlsx")
df_ety_prefix_suffix = df_ety_prefix_suffix[["search_word","word"]]
df_ety_prefix_suffix

,search_word,word
0,abaküs,abaküs
1,abluka,abluka
2,abluka,ablukaya
3,abluka,ablukayı
4,absorbe,absorbe
...,...,...
4753,şövalye,şövalyenin
4754,şövalye,şövalyesi
4755,şırınga,şırınga
4756,şırınga,şırıngayla


In [75]:
if file_ext == "3":
    df_all_word = pd.concat([df_word_prefix_suffix,df_ety_prefix_suffix],axis=0)
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
elif file_ext == "4":
    df_all_word = df_ety_prefix_suffix
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
elif file_ext == "5":
    df_all_word = df_word_prefix_suffix
    df_all_word.drop_duplicates(inplace=True)
    df_all_word.reset_index(drop=True, inplace=True)
else:
    pass
    
df_all_word

,search_word,word
0,ama,ama
1,bana,bana
2,ben,ben
3,ben,bence
4,ben,bende
...,...,...
162,şey,şeyler
163,şey,şeylerden
164,şey,şeylere
165,şey,şeyleri


In [76]:
df_all_word.search_word.nunique()

28

In [77]:
df_all_word.word.nunique()

155

In [78]:
df_shared_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/4-Shared Word/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency{file_ext}.xlsx")
#df_shared_all = pd.read_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Result_With_Frequency.xlsx")
df_shared_all

,word,frequency,twogram,freq_twogram,threegram,freq_threegram,fourgram,freq_fourgram,fivegram,freq_fivegram,sentence,freq_sentence
0,bir,18835735.0,bir şey,859944.0,bir şey yok,113165,bir şey var mı,41773.0,senin için bir şeyim var,2283.0,ne var,62532.0
1,bu,11062659.0,değil mi,585879.0,bir şey var,110455,bir şey daha var,12185.0,evet onun gibi bir şey,1046.0,ben de,59972.0
2,ne,8025880.0,ben de,377765.0,bu da ne,89463,bir şey mi var,11834.0,bir şey varsa o da,1010.0,değil mi,58386.0
3,ve,7766036.0,var mı,281902.0,bir şey değil,68498,ne var ne yok,9276.0,bir şey yok değil mi,893.0,bu da ne,50098.0
4,için,5484109.0,ne kadar,237876.0,bir şey mi,64933,bir şey değil mi,5080.0,o kadar çok şey var,790.0,ben mi,33652.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16922,NaN,NaN,NaN,NaN,bu onu bunu,10,NaN,NaN,NaN,NaN,NaN,NaN
16923,NaN,NaN,NaN,NaN,bu ona bu,10,NaN,NaN,NaN,NaN,NaN,NaN
16924,NaN,NaN,NaN,NaN,o şey kadar,10,NaN,NaN,NaN,NaN,NaN,NaN
16925,NaN,NaN,NaN,NaN,varsa beni bir,10,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
df_word_order_twogram = word_in_wordgroup(df_shared_all, "word", "twogram")
df_word_order_threegram = word_in_wordgroup(df_shared_all, "word", "threegram") 
df_word_order_fourgram = word_in_wordgroup(df_shared_all, "word", "fourgram") 
df_word_order_fivegram = word_in_wordgroup(df_shared_all, "word", "fivegram")
df_word_order_sentence = word_in_wordgroup(df_shared_all, "word", "sentence") 

In [80]:
df_word_order_twogram = pd.merge(df_word_order_twogram,df_all_word, how="inner", on="word")
df_word_order_threegram = pd.merge(df_word_order_threegram,df_all_word, how="inner", on="word")
df_word_order_fourgram = pd.merge(df_word_order_fourgram,df_all_word, how="inner", on="word")
df_word_order_fivegram = pd.merge(df_word_order_fivegram,df_all_word, how="inner", on="word")
df_word_order_sentence = pd.merge(df_word_order_sentence,df_all_word, how="inner", on="word")

In [81]:
df_word_order_join_twogram = df_word_order_twogram.groupby(["search_word"])["twogram"].apply(", ".join).reset_index()   # df_word_order_11.groupby(["word"])[f"twogram_pair_{lang_pair.lower()}"].transform(lambda x: ','.join(x))
df_word_order_join_threegram = df_word_order_threegram.groupby(["search_word"])["threegram"].apply(", ".join).reset_index()
df_word_order_join_fourgram = df_word_order_fourgram.groupby(["search_word"])["fourgram"].apply(", ".join).reset_index()
df_word_order_join_fivegram = df_word_order_fivegram.groupby(["search_word"])["fivegram"].apply(", ".join).reset_index()
df_word_order_join_sentence = df_word_order_sentence.groupby(["search_word"])["sentence"].apply(", ".join).reset_index()

In [82]:
dfs = [df_word_order_join_twogram,df_word_order_join_threegram,df_word_order_join_fourgram,df_word_order_join_fivegram,df_word_order_join_sentence]

In [83]:
df_word_order_join_all = reduce(lambda  left,right: pd.merge(left,right, on=['search_word'], how='outer'), dfs)  # left,right make left to right merge
#df_word_order_join_all = reduce(lambda  right,left: pd.merge(left,right, on=['word'], how='outer'), dfs)  # right,left make right to left merge
df_word_order_join_all.rename(columns={"search_word":"word"}, inplace=True)
df_word_order_join_all

,word,twogram,threegram,fourgram,fivegram,sentence
0,ama,"ama bu, ama ben, evet ama, ama o, ama sen, ama...","çok ama çok, ama bir şey, ama değil mi, evet a...","ama o kadar da, bir şey değil ama, bir şey yok...","ama bir şey daha var, ama o kadar da değil, am...","evet ama, ama ben, ama ne, ama bu, ama sen, ha..."
1,bana,"bana bir, bana ne, bunu bana, onu bana, bana d...","bana bir şey, sen de bana, bir de bana, bana b...","bana bir şey mi, ve sen de bana, evet bir de b...","bana bir şey var mı, bana da biraz var mı, sen...","bana mı, bana ne, bana da, bana değil, bu bana..."
2,ben,"ben de, ben bir, ve ben, ben mi, ama ben, hayı...","ben de seni, sen ve ben, ve ben de, evet ben d...","ben de senin gibi, ben senin gibi değilim, ben...","ne sen ne de ben, ben de senin gibi bir, sen m...","ben de, ben mi, hayır ben, evet ben de, ben de..."
3,beni,"beni mi, beni de, bu beni, sen beni, ve beni, ...","sen de beni, beni bir daha, o da beni, bu da b...","beni bunun için mi, ve o da beni, ve sen de be...","ne seni ne de beni, beni buraya bunun için mi,...","beni mi, beni de, beni değil, evet beni de, se..."
4,bir,"bir şey, bu bir, bir şeyler, bir daha, ben bir...","bir şey yok, bir şey var, bir şey değil, bir ş...","bir şey var mı, bir şey daha var, bir şey mi v...","senin için bir şeyim var, evet onun gibi bir ş...","bir şey değil, yok bir şey, bir şey yok, bir ş..."
5,bu,"bu kadar, bu da, bu çok, bu bir, bu ne, bu ben...","bu da ne, bu kadar mı, bu senin için, bu da ne...","bu da neyin nesi, bu benim için çok, bu şey de...","bu da bir şey mi, bu daha bir şey değil, ve bu...","bu da ne, bu ne, nedir bu, bu nedir, bu o, bu ..."
6,bunu,"bunu bana, bunu daha, bunu bir, ama bunu, bunu...","ben de bunu, bunu benim için, bunu bir daha, b...","bunu daha ne kadar, ve sen de bunu, ve bunu se...","bunu sen de ben de, bunu sen de benim kadar, s...","bunu da, ve bunu, ben bunu, bunu değil, ve bun..."
7,da,"bu da, da ne, o da, kadar da, daha da, da neyd...","bu da ne, o kadar da, ne kadar da, o da ne, bu...","bu da neyin nesi, o kadar da değil, biri varsa...","bir şey varsa o da, da onun gibi bir şey, biri...","bu da ne, o da ne, bu da neydi, o da neydi, bu..."
8,daha,"bir daha, biraz daha, daha çok, çok daha, daha...","bir şey daha, daha ne kadar, şey daha var, sen...","bir şey daha var, ve bir şey daha, daha ne kad...","ve bir şey daha var, ama bir şey daha var, bu ...","daha değil, bir şey daha var, biraz daha, bir ..."
9,de,"ben de, sen de, bir de, de seni, benim de, de ...","ben de seni, ve ben de, evet ben de, ve sen de...","ben de senin gibi, bu şey de ne, ben de senin ...","ne sen ne de ben, ben de senin gibi bir, ve bi...","ben de, sen de, bence de, benim de, evet ben d..."


In [84]:
# Option For Word Frequency
if shared_word_frequency:
    df_word_order_join_all = pd.merge(df_word_order_join_all,df_word_all, how="left", on="word")
    df_word_order_join_all.drop_duplicates(inplace=True)
    df_word_order_join_all = df_word_order_join_all.loc[:,["word","frequency","twogram","threegram","fourgram","fivegram","sentence"]]
    df_word_order_join_all.sort_values(by="frequency", inplace=True, ascending=False)
    df_word_order_join_all.reset_index(drop=True, inplace=True)
else:
    pass

df_word_order_join_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,bir,18835735,"bir şey, bu bir, bir şeyler, bir daha, ben bir...","bir şey yok, bir şey var, bir şey değil, bir ş...","bir şey var mı, bir şey daha var, bir şey mi v...","senin için bir şeyim var, evet onun gibi bir ş...","bir şey değil, yok bir şey, bir şey yok, bir ş..."
1,bu,11062659,"bu kadar, bu da, bu çok, bu bir, bu ne, bu ben...","bu da ne, bu kadar mı, bu senin için, bu da ne...","bu da neyin nesi, bu benim için çok, bu şey de...","bu da bir şey mi, bu daha bir şey değil, ve bu...","bu da ne, bu ne, nedir bu, bu nedir, bu o, bu ..."
2,ne,8025880,"ne kadar, ne var, da ne, bu ne, burada ne, sen...","bu da ne, ne kadar da, o da ne, daha ne kadar,...","ne var ne yok, senin için ne kadar, bu şey de ...","sende ne var ne yok, bunun senin için ne kadar...","ne var, bu da ne, ne için, bu ne, ne kadar, ne..."
3,ve,7766036,"ve ben, ve bu, ve bir, sen ve, ve sen, ve o, v...","sen ve ben, ve ben de, ve sen de, ve bu da, ve...","ve bir şey daha, sen ve ben birlikte, ve sen d...","ve bir şey daha var, ve bu da senin için, ve b...","sen ve ben, ve sen, ve ben, ve bu, ve ne, ve b..."
4,için,5484109,"senin için, benim için, için bir, bunun için, ...","benim için bir, senin için bir, bu senin için,...","benim için bir şey, senin için bir şey, bu ben...","senin için bir şeyim var, senin için bir şey v...","ne için, senin için, bu senin için, benim için..."
5,mi,5362714,"değil mi, şey mi, ben mi, için mi, sen mi, mi ...","bir şey mi, yok değil mi, var değil mi, şey mi...","bir şey mi var, bir şey değil mi, evet mi hayı...","bir şey yok değil mi, bir şey var değil mi, o ...","değil mi, ben mi, sen mi, benim mi, evet mi, b..."
6,o,5013838,"o kadar, o da, o bir, o benim, bu o, o ne, ama...","o kadar da, o da ne, o kadar çok, o da neydi, ...","o kadar da değil, biri varsa o da, o kadar çok...","bir şey varsa o da, o kadar çok şey var, biri ...","bu o, nedir o, o da ne, o ne, neymiş o, o beni..."
7,ben,4908913,"ben de, ben bir, ve ben, ben mi, ama ben, hayı...","ben de seni, sen ve ben, ve ben de, evet ben d...","ben de senin gibi, ben senin gibi değilim, ben...","ne sen ne de ben, ben de senin gibi bir, sen m...","ben de, ben mi, hayır ben, evet ben de, ben de..."
8,de,4880315,"ben de, sen de, bir de, de seni, benim de, de ...","ben de seni, ve ben de, evet ben de, ve sen de...","ben de senin gibi, bu şey de ne, ben de senin ...","ne sen ne de ben, ben de senin gibi bir, ve bi...","ben de, sen de, bence de, benim de, evet ben d..."
9,çok,4852169,"bu çok, için çok, daha çok, çok daha, çok şey,...","o kadar çok, bu kadar çok, benim için çok, çok...","bu benim için çok, o kadar çok şey, bu senin i...","o kadar çok şey var, ne kadar çok o kadar, o k...","bu çok, çok değil, daha çok, evet çok, sen çok..."


In [85]:
df_word_order_join_all.word.nunique()

28

In [86]:
df_word_order_join_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency{file_ext}3.xlsx", index=False)

#### Copy Move And Delete

In [87]:
output_file2 = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*{file_ext}3.xlsx")
output_file2

['Turkish_Spanish_Shared_Join_Result_Without_Frequency53.xlsx']

In [88]:
for l in output_file2:
    source = l # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [89]:
for j in output_file2:
    try:
        os.remove(j)
    except:
        pass

### Prefix Suffix Shared File Word Result Select

In [53]:
import os
import multiprocessing
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil

In [90]:
## language pair (same previous part parameter)
#lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "French"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# word sample
word_sample_num = 20

print(f"{file_ext}")

5


In [91]:
def lower_strip_func(x):
    try:
        var_low = x.lower()
        var_out = var_low.strip()
    except:
        var_out = x
    return var_out 

In [92]:
df_shared_process_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}/{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Result_Without_Frequency{file_ext}3.xlsx")
df_shared_process_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,bir,18835735,"bir şey, bu bir, bir şeyler, bir daha, ben bir...","bir şey yok, bir şey var, bir şey değil, bir ş...","bir şey var mı, bir şey daha var, bir şey mi v...","senin için bir şeyim var, evet onun gibi bir ş...","bir şey değil, yok bir şey, bir şey yok, bir ş..."
1,bu,11062659,"bu kadar, bu da, bu çok, bu bir, bu ne, bu ben...","bu da ne, bu kadar mı, bu senin için, bu da ne...","bu da neyin nesi, bu benim için çok, bu şey de...","bu da bir şey mi, bu daha bir şey değil, ve bu...","bu da ne, bu ne, nedir bu, bu nedir, bu o, bu ..."
2,ne,8025880,"ne kadar, ne var, da ne, bu ne, burada ne, sen...","bu da ne, ne kadar da, o da ne, daha ne kadar,...","ne var ne yok, senin için ne kadar, bu şey de ...","sende ne var ne yok, bunun senin için ne kadar...","ne var, bu da ne, ne için, bu ne, ne kadar, ne..."
3,ve,7766036,"ve ben, ve bu, ve bir, sen ve, ve sen, ve o, v...","sen ve ben, ve ben de, ve sen de, ve bu da, ve...","ve bir şey daha, sen ve ben birlikte, ve sen d...","ve bir şey daha var, ve bu da senin için, ve b...","sen ve ben, ve sen, ve ben, ve bu, ve ne, ve b..."
4,için,5484109,"senin için, benim için, için bir, bunun için, ...","benim için bir, senin için bir, bu senin için,...","benim için bir şey, senin için bir şey, bu ben...","senin için bir şeyim var, senin için bir şey v...","ne için, senin için, bu senin için, benim için..."
5,mi,5362714,"değil mi, şey mi, ben mi, için mi, sen mi, mi ...","bir şey mi, yok değil mi, var değil mi, şey mi...","bir şey mi var, bir şey değil mi, evet mi hayı...","bir şey yok değil mi, bir şey var değil mi, o ...","değil mi, ben mi, sen mi, benim mi, evet mi, b..."
6,o,5013838,"o kadar, o da, o bir, o benim, bu o, o ne, ama...","o kadar da, o da ne, o kadar çok, o da neydi, ...","o kadar da değil, biri varsa o da, o kadar çok...","bir şey varsa o da, o kadar çok şey var, biri ...","bu o, nedir o, o da ne, o ne, neymiş o, o beni..."
7,ben,4908913,"ben de, ben bir, ve ben, ben mi, ama ben, hayı...","ben de seni, sen ve ben, ve ben de, evet ben d...","ben de senin gibi, ben senin gibi değilim, ben...","ne sen ne de ben, ben de senin gibi bir, sen m...","ben de, ben mi, hayır ben, evet ben de, ben de..."
8,de,4880315,"ben de, sen de, bir de, de seni, benim de, de ...","ben de seni, ve ben de, evet ben de, ve sen de...","ben de senin gibi, bu şey de ne, ben de senin ...","ne sen ne de ben, ben de senin gibi bir, ve bi...","ben de, sen de, bence de, benim de, evet ben d..."
9,çok,4852169,"bu çok, için çok, daha çok, çok daha, çok şey,...","o kadar çok, bu kadar çok, benim için çok, çok...","bu benim için çok, o kadar çok şey, bu senin i...","o kadar çok şey var, ne kadar çok o kadar, o k...","bu çok, çok değil, daha çok, evet çok, sen çok..."


In [93]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all = df_word_all.loc[:,["word","frequency"]]
df_word_all["word"] = df_word_all["word"].apply(lambda x: lower_strip_func(x))
df_word_all

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
...,...,...
988212,karneleme,5
988213,karnaya,5
988214,dörtlümüzün,5
988215,karnavalınız,5


In [94]:
df_twogram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Twogram_Merge.csv")
df_twogram_all = df_twogram_all.loc[:,["twogram","frequency"]]
df_twogram_all["twogram"] = df_twogram_all["twogram"].apply(lambda x: lower_strip_func(x))
df_twogram_all

,twogram,frequency
0,bir şey,859944
1,değil mi,585879
2,ben de,377765
3,teşekkür ederim,370619
4,ne oldu,322758
...,...,...
4457029,fikret cibran,3
4457030,romalı fikret,3
4457031,fikret ciooney,3
4457032,fikret cisco,3


In [95]:
df_threegram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Threegram_Merge.csv")
df_threegram_all = df_threegram_all.loc[:,["threegram","frequency"]]
df_threegram_all["threegram"] = df_threegram_all["threegram"].apply(lambda x: lower_strip_func(x))
df_threegram_all

,threegram,frequency
0,bir şey yok,113165
1,bir şey var,110455
2,bu da ne,89463
3,her şey yolunda,75968
4,başka bir şey,75193
...,...,...
3009751,haydi büyük fikret,5
3009752,fikret caesardan beri,5
3009753,fikret cage kazandı,5
3009754,haydi bakayım fikret,5


In [96]:
df_fourgram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fourgram_Merge.csv")
df_fourgram_all = df_fourgram_all.loc[:,["fourgram","frequency"]]
df_fourgram_all["fourgram"] = df_fourgram_all["fourgram"].apply(lambda x: lower_strip_func(x))
df_fourgram_all

,fourgram,frequency
0,bir şey var mı,41773
1,her şey yolunda mı,31126
2,burada ne işin var,21993
3,bir sorun mu var,21423
4,ben de seni seviyorum,17338
...,...,...
3052209,fikret miloya merhaba de,5
3052210,fikret millsin oğlu mu,5
3052211,fikret millet iyi iş,5
3052212,fikret millet bize bakıyor,5


In [97]:
df_fivegram_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/N Gram/Merge/Fivegram_Merge.csv")
df_fivegram_all = df_fivegram_all.loc[:,["fivegram","frequency"]]
df_fivegram_all["fivegram"] = df_fivegram_all["fivegram"].apply(lambda x: lower_strip_func(x))
df_fivegram_all

,fivegram,frequency
0,başka bir şey var mı,14104
1,bu da ne demek oluyor,10205
2,o kadar da kötü değil,7012
3,sence de öyle değil mi,6305
4,sana bir şey sorabilir miyim,6224
...,...,...
1096831,peder fikret hep şöyle söylerdi,4
1096832,peder fikret intihar etmeye çalışıyor,4
1096833,fikret dolson 12 gün yaşadı,4
1096834,ama fikret diye biri yoktu,4


In [98]:
df_sentence_all = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Result/Sentence/Merge/Sentence_Merge.csv")
df_sentence_all = df_sentence_all.loc[:,["sentence","frequency"]]
df_sentence_all["sentence"] = df_sentence_all["sentence"].apply(lambda x: lower_strip_func(x))
df_sentence_all

,sentence,frequency
0,evet,1948596
1,fikret,1533918
2,hayır,1250401
3,tamam,882921
4,ne,753105
...,...,...
2913965,hayır ben bir pikap kamyon alacağım,6
2913966,eminim uydurmuştur,6
2913967,hemen bir kement ile onu yakalar ve aşağı çekerim,6
2913968,tabii gerçek bir profesyonel o,6


In [99]:
for i in range(len(df_shared_process_all)):
    # column result
    try:
        # column result
        df_two_var = pd.DataFrame(df_shared_process_all.loc[i,"twogram"].split(", "), columns=["twogram"])
        # merge with all
        df_two_var_merge = pd.merge(df_two_var, df_twogram_all, how="left", on="twogram")
        df_two_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_two_var_merge_select = df_two_var_merge.head(word_sample_num)
        # to list value
        df_two_var_list = df_two_var_merge_select["twogram"].to_list()
        # list join
        df_two_var_list_join = ", ".join(df_two_var_list)
        # change value
        df_shared_process_all.loc[i,"twogram"] = df_two_var_list_join
    except:
        pass
    try:
        # column result
        df_three_var = pd.DataFrame(df_shared_process_all.loc[i,"threegram"].split(", "), columns=["threegram"])
        # merge with all
        df_three_var_merge = pd.merge(df_three_var, df_threegram_all, how="left", on="threegram")
        df_three_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_three_var_merge_select = df_three_var_merge.head(word_sample_num)
        # to list value
        df_three_var_list = df_three_var_merge_select["threegram"].to_list()
        # list join
        df_three_var_list_join = ", ".join(df_three_var_list)
        # change value
        df_shared_process_all.loc[i,"threegram"] = df_three_var_list_join
    except:
        pass
    try:
        # column result
        df_four_var = pd.DataFrame(df_shared_process_all.loc[i,"fourgram"].split(", "), columns=["fourgram"])
        # merge with all
        df_four_var_merge = pd.merge(df_four_var, df_fourgram_all, how="left", on="fourgram")
        df_four_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_four_var_merge_select = df_four_var_merge.head(word_sample_num)
        # to list value
        df_four_var_list = df_four_var_merge_select["fourgram"].to_list()
        # list join
        df_four_var_list_join = ", ".join(df_four_var_list)
        # change value
        df_shared_process_all.loc[i,"fourgram"] = df_four_var_list_join
    except:
        pass
    try:
        # column result
        df_five_var = pd.DataFrame(df_shared_process_all.loc[i,"fivegram"].split(", "), columns=["fivegram"])
        # merge with all
        df_five_var_merge = pd.merge(df_five_var, df_fivegram_all, how="left", on="fivegram")
        df_five_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_five_var_merge_select = df_five_var_merge.head(word_sample_num)
        # to list value
        df_five_var_list = df_five_var_merge_select["fivegram"].to_list()
        # list join
        df_five_var_list_join = ", ".join(df_five_var_list)
        # change value
        df_shared_process_all.loc[i,"fivegram"] = df_five_var_list_join
    except:
        pass
    try:
        # column result
        df_sentence_var = pd.DataFrame(df_shared_process_all.loc[i,"sentence"].split(", "), columns=["sentence"])
        # merge with all
        df_sentence_var_merge = pd.merge(df_sentence_var, df_sentence_all, how="left", on="sentence")
        df_sentence_var_merge.sort_values(by="frequency", ascending=False, inplace=True)
        df_sentence_var_merge_select = df_sentence_var_merge.head(word_sample_num)
        # to list value
        df_sentence_var_list = df_sentence_var_merge_select["sentence"].to_list()
        # list join
        df_sentence_var_list_join = ", ".join(df_sentence_var_list)
        # change value
        df_shared_process_all.loc[i,"sentence"] = df_sentence_var_list_join
    except:
        pass      

df_shared_process_all

,word,frequency,twogram,threegram,fourgram,fivegram,sentence
0,bir,18835735,"bir şey, bu bir, bir şeyler, bir daha, ben bir...","bir şey yok, bir şey var, bir şey değil, bir ş...","bir şey var mı, bir şey daha var, bir şey mi v...","senin için bir şeyim var, evet onun gibi bir ş...","bir şey değil, yok bir şey, bir şey yok, bir ş..."
1,bu,11062659,"bu kadar, bu da, bu çok, bu bir, bu ne, burada...","bu da ne, bu kadar mı, bu senin için, bu da ne...","bu da neyin nesi, bu benim için çok, burada bi...","bunun senin için ne kadar, bunun benim için ne...","bu da ne, bu ne, nedir bu, bu nedir, bu o, bu ..."
2,ne,8025880,"ne kadar, ne var, da ne, bu ne, burada ne, sen...","bu da ne, ne kadar da, o da ne, daha ne kadar,...","ne var ne yok, bu da neyin nesi, bu da neyin n...","sende ne var ne yok, bunun senin için ne kadar...","ne var, bu da ne, ne için, bu ne, nedir bu, bu..."
3,ve,7766036,"ve ben, ve bu, ve bir, sen ve, ve sen, ve o, v...","sen ve ben, ve ben de, ve sen de, ve bu da, ve...","ve bir şey daha, sen ve ben birlikte, ve sen d...","ve bir şey daha var, ve bu da senin için, ve b...","sen ve ben, ve sen, ve ben, ve bu, ve ne, ve b..."
4,için,5484109,"senin için, benim için, için bir, bunun için, ...","benim için bir, senin için bir, bu senin için,...","benim için bir şey, senin için bir şey, bu ben...","senin için bir şeyim var, senin için bir şey v...","ne için, senin için, bu senin için, benim için..."
5,mi,5362714,"değil mi, şey mi, ben mi, için mi, sen mi, mi ...","bir şey mi, yok değil mi, var değil mi, şey mi...","bir şey mi var, bir şey değil mi, evet mi hayı...","bir şey yok değil mi, bir şey var değil mi, o ...","değil mi, ben mi, sen mi, sen misin, benim mi,..."
6,o,5013838,"o kadar, o da, o bir, o benim, bu o, onun için...","o kadar da, o da ne, o kadar çok, ben de onu, ...","onun gibi bir şey, o kadar da değil, biri vars...","evet onun gibi bir şey, bir şey varsa o da, o ...","bu o, nedir o, o da ne, o ne, neymiş o, o beni..."
7,ben,4908913,"ben de, benim için, bu benim, ben bir, ve ben,...","ben de seni, sen ve ben, benim için bir, ve be...","benim için bir şey, bu benim için çok, ben de ...","biri varsa o da benim, bunun benim için ne kad...","ben de, ben mi, bence de, evet benim, o benim,..."
8,de,4880315,"ben de, sen de, bir de, de seni, benim de, de ...","ben de seni, ve ben de, evet ben de, ve sen de...","ben de senin gibi, bu şey de ne, ben de senin ...","ne sen ne de ben, ben de senin gibi bir, ve bi...","ben de, sen de, bence de, benim de, evet ben d..."
9,çok,4852169,"bu çok, için çok, daha çok, çok daha, çok şey,...","o kadar çok, bu kadar çok, benim için çok, çok...","bu benim için çok, o kadar çok şey, bu senin i...","o kadar çok şey var, ne kadar çok o kadar, o k...","bu çok, çok değil, daha çok, evet çok, sen çok..."


In [100]:
#df_shared_process_all.sort_values(by="frequency", ascending=False, inplace=True)

In [101]:
df_shared_process_all.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared_Join_Select_Result_Without_Frequency{file_ext}4.xlsx", index=False)

#### Copy Move And Delete

In [102]:
output_file3 = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_Shared*_Select_*{file_ext}4.xlsx")
output_file3

['Turkish_Spanish_Shared_Join_Select_Result_Without_Frequency54.xlsx']

In [103]:
for l in output_file3:
    source = l # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [104]:
for j in output_file3:
    try:
        os.remove(j)
    except:
        pass

### Word Count Analysis

In [45]:
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil

In [46]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

In [47]:
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}").mkdir(parents=True, exist_ok=True)
Path(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/Deploy Result Manuel").mkdir(parents=True, exist_ok=True)

In [ ]:
def word_count_result(df,column_list): # df is dataframe, column_list is list value
    '''
    word_count_bool(df, column_list): df columns word count for word frequency\n
    df is dataframe, column_list is list value\n
    word_count_bool(df, ["word","twogram"]):
    '''
    list_all = []
    for i in df.loc[:,[x for x in column_list]].columns:
        var_list = df[f"{i}"].dropna().tolist()
        for j in var_list:
            list_all.append(j)
    text = " ".join(list_all)
    word_list = re.findall(r"\w+",text, re.UNICODE)
    df_word_list = pd.DataFrame(word_list, columns=["word"])
    #df_word_list.rename(columns={0:"word"}, inplace=True)
    df_word_count = pd.DataFrame(df_word_list.value_counts())
    df_word_count.reset_index(inplace=True)
    df_word_count.rename(columns={0:"word_count"}, inplace=True)
    
    return  df_word_count

#### Hybrid Words

In [63]:
df_hybrid = pd.read_excel("Turkish English manual selected 2 gram hybrids 3.xlsx", sheet_name="2 gram hybrid")
df_hybrid

,word,twogram_pair1,twogram_pair2,twogram_pair3,twogram_pair4
0,bir,bir doktor,bir polis,bir dolar,bir telefon
1,bu,bu doktor,bu komik,bu telefon,bu bebek
2,ne,ne şans,mesaj ne,plan ne,ne sürpriz
3,ve,ve müzik,ve şeker,ve patates,ve çikolata
4,için,kontrol için,bebek için,dans için,film için
5,mi,komik mi,normal mi,problem mi,profesyonel mi
6,o,o ajan,o müzik,o park,o film
7,ben,ben senatör,ben ajan,ben general,ben profesör
8,de,patates de,şef de,profesör de,prenses de
9,çok,çok komik,çok normal,çok pardon,çok süper


In [64]:
df_hybrid_count = word_count_result(df_hybrid, ["twogram_pair1","twogram_pair2","twogram_pair3","twogram_pair4"])
df_hybrid_count

,word,word_count
0,bebek,6
1,komik,6
2,normal,5
3,sen,4
4,onu,4
...,...,...
89,okula,1
90,park,1
91,parka,1
92,pizza,1


In [65]:
df_hybrid_count_merge = pd.merge(df_hybrid,df_hybrid_count,how="left",on="word")
df_hybrid_count_merge

,word,twogram_pair1,twogram_pair2,twogram_pair3,twogram_pair4,word_count
0,bir,bir doktor,bir polis,bir dolar,bir telefon,4
1,bu,bu doktor,bu komik,bu telefon,bu bebek,4
2,ne,ne şans,mesaj ne,plan ne,ne sürpriz,4
3,ve,ve müzik,ve şeker,ve patates,ve çikolata,4
4,için,kontrol için,bebek için,dans için,film için,4
5,mi,komik mi,normal mi,problem mi,profesyonel mi,4
6,o,o ajan,o müzik,o park,o film,4
7,ben,ben senatör,ben ajan,ben general,ben profesör,4
8,de,patates de,şef de,profesör de,prenses de,4
9,çok,çok komik,çok normal,çok pardon,çok süper,4


In [67]:
df_hybrid_count_merge2 = pd.merge(df_hybrid,df_hybrid_count,how="outer",on="word")
df_hybrid_count_merge2

,word,twogram_pair1,twogram_pair2,twogram_pair3,twogram_pair4,word_count
0,bir,bir doktor,bir polis,bir dolar,bir telefon,4
1,bu,bu doktor,bu komik,bu telefon,bu bebek,4
2,ne,ne şans,mesaj ne,plan ne,ne sürpriz,4
3,ve,ve müzik,ve şeker,ve patates,ve çikolata,4
4,için,kontrol için,bebek için,dans için,film için,4
...,...,...,...,...,...,...
89,okula,NaN,NaN,NaN,NaN,1
90,park,NaN,NaN,NaN,NaN,1
91,parka,NaN,NaN,NaN,NaN,1
92,pizza,NaN,NaN,NaN,NaN,1


In [73]:
writer = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_Hybrid_Word_Count.xlsx", engine='xlsxwriter')

In [74]:
df_hybrid_count_merge.to_excel(writer, sheet_name='28_Hybrid_Word_Count', index=False)
df_hybrid_count_merge2.to_excel(writer, sheet_name='All_Hybrid_Word_Count', index=False)

In [75]:
writer.save()

#### Target Words

In [69]:
df_target = pd.read_excel("Turkish English manual selected 2 gram hybrids 3.xlsx", sheet_name="2 gram target")
df_target

,word,twogram_1,twogram_2,twogram_3,twogram_4
0,bir,bir şey,bir daha,bir de,bir çok
1,bu,bu kadar,bu ne,bu çok,bu değil
2,ne,ne kadar,ne için,bana ne,ne gibi
3,ve,ve de,ve evet,ve ne,ve sen
4,için,ne için,benim için,senin için,onun için
5,mi,değil mi,ben mi,sen mi,evet mi
6,o,o kadar,o da,o değil,evet o
7,ben,ben de,ben değil,hayır ben,ben mi
8,de,sen de,bir de,beni de,beni de
9,çok,bu çok,daha çok,çok var,çok şey


In [70]:
df_target_count = word_count_result(df_target, ["twogram_1","twogram_2","twogram_3","twogram_4"])
df_target_count

,word,word_count
0,evet,14
1,değil,14
2,hayır,13
3,bu,12
4,ne,11
5,o,10
6,de,10
7,daha,9
8,var,9
9,bana,8


In [71]:
df_target_count_merge = pd.merge(df_target,df_target_count,how="left",on="word")
df_target_count_merge

,word,twogram_1,twogram_2,twogram_3,twogram_4,word_count
0,bir,bir şey,bir daha,bir de,bir çok,7
1,bu,bu kadar,bu ne,bu çok,bu değil,12
2,ne,ne kadar,ne için,bana ne,ne gibi,11
3,ve,ve de,ve evet,ve ne,ve sen,5
4,için,ne için,benim için,senin için,onun için,6
5,mi,değil mi,ben mi,sen mi,evet mi,8
6,o,o kadar,o da,o değil,evet o,10
7,ben,ben de,ben değil,hayır ben,ben mi,7
8,de,sen de,bir de,beni de,beni de,10
9,çok,bu çok,daha çok,çok var,çok şey,7


In [72]:
df_target_count_merge2 = pd.merge(df_target,df_target_count,how="outer",on="word")
df_target_count_merge2

,word,twogram_1,twogram_2,twogram_3,twogram_4,word_count
0,bir,bir şey,bir daha,bir de,bir çok,7
1,bu,bu kadar,bu ne,bu çok,bu değil,12
2,ne,ne kadar,ne için,bana ne,ne gibi,11
3,ve,ve de,ve evet,ve ne,ve sen,5
4,için,ne için,benim için,senin için,onun için,6
5,mi,değil mi,ben mi,sen mi,evet mi,8
6,o,o kadar,o da,o değil,evet o,10
7,ben,ben de,ben değil,hayır ben,ben mi,7
8,de,sen de,bir de,beni de,beni de,10
9,çok,bu çok,daha çok,çok var,çok şey,7


In [76]:
writer2 = pd.ExcelWriter(f"{lang_folder}_{lang_pair}_Target_Word_Count.xlsx", engine='xlsxwriter')

In [77]:
df_target_count_merge.to_excel(writer2, sheet_name='28_Target_Word_Count', index=False)
df_target_count_merge2.to_excel(writer2, sheet_name='All_Target_Word_Count', index=False)

In [78]:
writer2.save()

#### All Target Hybrid Word Count 

In [79]:
df_target_count

,word,word_count
0,evet,14
1,değil,14
2,hayır,13
3,bu,12
4,ne,11
5,o,10
6,de,10
7,daha,9
8,var,9
9,bana,8


In [80]:
df_hybrid_count

,word,word_count
0,bebek,6
1,komik,6
2,normal,5
3,sen,4
4,onu,4
...,...,...
89,okula,1
90,park,1
91,parka,1
92,pizza,1


In [82]:
df_all_word = pd.concat([df_target_count, df_hybrid_count], axis=0)
df_all_word

,word,word_count
0,evet,14
1,değil,14
2,hayır,13
3,bu,12
4,ne,11
...,...,...
89,okula,1
90,park,1
91,parka,1
92,pizza,1


In [87]:
df_all_word.groupby("word")[["word_count"]].sum().reset_index(inplace=True)
df_all_word.sort_values(by="word_count", ascending=False, inplace=True)
df_all_word

,word,word_count
0,evet,14
1,değil,14
2,hayır,13
3,bu,12
4,ne,11
...,...,...
64,süper,1
65,problem,1
66,manyaksın,1
67,popüler,1


In [89]:
df_all_word.to_excel(f"{lang_folder}_{lang_pair}_Target_Hybrid_Word_Count.xlsx", index=False)

#### Copy Move And Delete

In [4]:
output_file4 = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_*_Word_Count.xlsx")
output_file4

['Turkish_English_Hybrid_Word_Count.xlsx',
 'Turkish_English_Target_Word_Count.xlsx',
 'Turkish_English_Target_Hybrid_Word_Count.xlsx']

In [7]:
for o in output_file4:
    source = o # source directory
    destination = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/Deployment/Result/5-Shared Word File/{lang_folder.capitalize()} {lang_pair.capitalize()}"
    shutil.copy2(source, destination)

In [8]:
for p in output_file4:
    try:
        os.remove(p)
    except:
        pass